IPD single-cell genomics workshop - May 2024

In this project you will be analyzing B cells from patients who have recovered from SARS-CoV2 infection. The data was generated using the 10x genomics platform.

The paper is "Maturation and persistence of the anti-SARS-CoV-2 memory B cell response"


(https://www.cell.com/cell/fulltext/S0092-8674(21)00093-3)

Questions include:
- What types of B cells do you observe?
- Are B cell states similar between individuals?
- Do B cell states change in late vs. early response to infection?
- What are the features of BCRs in late vs early response?
- Are there any shared BCR clones across individuals?
- What are the BCR features and gene expression states of expanded BCR clones?

In [ ]:
# Start with section to define shell call function and install packages
shell_call <- function(command, ...) {
  result <- system(command, intern = TRUE, ...)
  cat(paste0(result, collapse = "\n"))
}

loadPackages = function(pkgs){
  myrequire = function(...){
    suppressWarnings(suppressMessages(suppressPackageStartupMessages(require(...))))
  }
  ok = sapply(pkgs, require, character.only=TRUE, quietly=TRUE)
  if (!all(ok)){
    message("There are missing packages: ", paste(pkgs[!ok], collapse=", "))
  }
}

## Setup R2U
download.file("https://github.com/eddelbuettel/r2u/raw/master/inst/scripts/add_cranapt_jammy.sh",
              "add_cranapt_jammy.sh")
Sys.chmod("add_cranapt_jammy.sh", "0755")
shell_call("./add_cranapt_jammy.sh")
bspm::enable()
options(bspm.version.check=FALSE)
shell_call("rm add_cranapt_jammy.sh")

51 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
wget is already the newest version (1.21.2-2ubuntu1).
ca-certificates is already the newest version (20230311ubuntu0.22.04.1).
gnupg is already the newest version (2.2.27-3ubuntu2.1).
gnupg set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.
-----BEGIN PGP PUBLIC KEY BLOCK-----

...
I5rGSOk715L9
=VuA5
-----END PGP PUBLIC KEY BLOCK-----
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelea

Tracing function "install.packages" in package "utils"



In [ ]:
## Install the R packages
cranPkgs2Install = c("BiocManager")
install.packages(cranPkgs2Install, ask=FALSE, update=TRUE, quietly=TRUE)
system("sudo apt install libgsl-dev")
BiocManager::install("scRepertoire")
install.packages('Seurat')

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.19 (BiocManager 1.30.23), R 4.4.0 (2024-04-24)

Installing package(s) 'BiocVersion', 'scRepertoire'

Old packages: 'backports', 'bit', 'brio', 'broom', 'bslib', 'cachem', 'callr',
  'curl', 'data.table', 'DBI', 'dbplyr', 'devtools', 'digest', 'farver',
  'fastmap', 'fs', 'gargle', 'GenomeInfoDb', 'ggplot2', 'gh', 'gtable',
  'highr', 'htmltools', 'httr2', 'isoband', 'knitr', 'munsell', 'openssl',
  'pkgbuild', 'processx', 'ragg', 'remotes', 'rlang', 'rmarkdown',
  'rstudioapi', 'rvest', 'S4Arrays', 'sass', 'SparseArray', 'stringi',
  'systemfonts', 'testthat', 'textshaping', 'tidyselect', 'tinytex', 'usethis',
  'whisker', 'xfun', 'xopen', 'zip', 'codetools', 'KernSmooth', 'lattice',
  'MASS', 'Matrix', 'nlme', 'survival'



In [ ]:
# Download a Seurat object containing the gene expression data and metadata for two individuals (at two time points) from the paper
shell_call("wget -q --output-document Sokal21_Bcells.Rds https://www.dropbox.com/scl/fi/swv5419nijrsyx2ivmjnw/Sokal21_Bcells.Rds?rlkey=deb763oh199j3mclspfbgdvqw&dl=0")

In [ ]:
# Download the corresponding BCR contigs. Note that heavy chain (HC) and light chain (LC) are in separate files that we will merge later.
shell_call("wget -q --output-document S-CoV1_M0_Sort1_B_VDJ_HC_filtered.tsv.gz https://www.dropbox.com/scl/fi/owo1v2ua034s6s6rb046v/S-CoV1_M0_Sort1_B_VDJ_HC_filtered.tsv.gz?rlkey=l9ivl9k6q5bykgy8b9sdfnvo5&dl=0")
shell_call("wget -q --output-document S-CoV1_M0_Sort1_B_VDJ_LC_filtered.tsv.gz https://www.dropbox.com/scl/fi/u8wcn4ueu2xia3q5qoiab/S-CoV1_M0_Sort1_B_VDJ_LC_filtered.tsv.gz?rlkey=a3k028qfp6zznpvflcr6i7pyg&dl=0")

shell_call("wget -q --output-document S-CoV1_M6_Sort1_B_VDJ_HC_filtered.tsv.gz https://www.dropbox.com/scl/fi/ldsh5rml9o8dckja5le2c/S-CoV1_M6_Sort1_B_VDJ_HC_filtered.tsv.gz?rlkey=ul9cxksligcxbs3p1yv27sbe0&dl=0")
shell_call("wget -q --output-document S-CoV1_M6_Sort1_B_VDJ_LC_filtered.tsv.gz https://www.dropbox.com/scl/fi/x0c87vhriyi71tuqjmj2k/S-CoV1_M6_Sort1_B_VDJ_LC_filtered.tsv.gz?rlkey=0fuoesu6isnqhbzu5pcfh1v82&dl=0")

shell_call("wget -q --output-document S-CoV13_M0_Sort1_B_VDJ_HC_filtered.tsv.gz https://www.dropbox.com/scl/fi/y0e136iqbs2hr49eekwc0/S-CoV13_M0_Sort1_B_VDJ_HC_filtered.tsv.gz?rlkey=9an8qs67aqkmlqrnxabpcolyn&dl=0")
shell_call("wget -q --output-document S-CoV13_M0_Sort1_B_VDJ_LC_filtered.tsv.gz https://www.dropbox.com/scl/fi/1ibcepgrc2hswww3krzww/S-CoV13_M0_Sort1_B_VDJ_LC_filtered.tsv.gz?rlkey=5ukvzz74q4qgt2kv0cy3d8tvx&dl=0")

shell_call("wget -q --output-document S-CoV13_M6_Sort1_B_VDJ_HC_filtered.tsv.gz https://www.dropbox.com/scl/fi/xigcdobssfj4tcn05zc4o/S-CoV13_M6_Sort1_B_VDJ_HC_filtered.tsv.gz?rlkey=q3gikng60kmwr7udyl144xtjz&dl=0")
shell_call("wget -q --output-document S-CoV13_M6_Sort1_B_VDJ_LC_filtered.tsv.gz https://www.dropbox.com/scl/fi/5zn4xz26hgrvx5zg64tzw/S-CoV13_M6_Sort1_B_VDJ_LC_filtered.tsv.gz?rlkey=sgq7acfuyeqb7tbmyg7rr3on1&dl=0")


In [ ]:
# Load in Seurat object and begin exploring it
library(Seurat)
bcells = readRDS("Sokal21_Bcells.Rds")

In [ ]:
# Explore the Seurat object and make sure it has the expected number of features and samples
head(colnames(bcells))

[1] "S1_AAACCTGGTCTCGTTC-1" "S1_AAACCTGGTGATGCCC-1" "S1_AAACCTGTCTCACATT-1"
[4] "S1_AAACCTGTCTTGCATT-1" "S1_AAACGGGAGAACAATC-1" "S1_AAACGGGGTCTAGTCA-1"

In [ ]:
# Read in the BCR contigs
bcr_hc1 = read.table("S-CoV1_M0_Sort1_B_VDJ_HC_filtered.tsv.gz",sep="\t",header=T)
bcr_lc1 = read.table("S-CoV1_M0_Sort1_B_VDJ_LC_filtered.tsv.gz",sep="\t",header=T)
bcr_hc2 = read.table("S-CoV1_M6_Sort1_B_VDJ_HC_filtered.tsv.gz",sep="\t",header=T)
bcr_lc2 = read.table("S-CoV1_M6_Sort1_B_VDJ_LC_filtered.tsv.gz",sep="\t",header=T)
bcr_hc3 = read.table("S-CoV13_M0_Sort1_B_VDJ_HC_filtered.tsv.gz",sep="\t",header=T)
bcr_lc3 = read.table("S-CoV13_M0_Sort1_B_VDJ_LC_filtered.tsv.gz",sep="\t",header=T)
bcr_hc4 = read.table("S-CoV13_M6_Sort1_B_VDJ_HC_filtered.tsv.gz",sep="\t",header=T)
bcr_lc4 = read.table("S-CoV13_M6_Sort1_B_VDJ_LC_filtered.tsv.gz",sep="\t",header=T)

In [ ]:
# Merge the files into one file that we will use as input to scRepertoire
bcr_vdj_all = rbind(bcr_hc1,bcr_lc1,bcr_hc2,bcr_lc2,bcr_hc3,bcr_lc3,bcr_hc4,bcr_lc4)

In [ ]:
# Load scRepertoire and make the contig list
library(scRepertoire)
contig.list <- loadContigs(bcr_vdj_all, format = "AIRR")

Loading required package: ggplot2



In [ ]:
# One more issue - we have to make sure the cell barcodes are the same in the VDJ table and the Seurat object
# Check the barcode names in the Seurat object
head(colnames(bcells))

[1] "S1_AAACCTGGTCTCGTTC-1" "S1_AAACCTGGTGATGCCC-1" "S1_AAACCTGTCTCACATT-1"
[4] "S1_AAACCTGTCTTGCATT-1" "S1_AAACGGGAGAACAATC-1" "S1_AAACGGGGTCTAGTCA-1"

In [ ]:
# Check the barcode names in the contig list
head(contig.list[[1]]$barcode)

[1] "S1_AGCTCTCGTTTGCATG-1" "S1_ATAGACCCATGTCGAT-1" "S1_ATCTGCCCAGCCAATT-1"
[4] "S1_CAAGGCCGTGTGAAAT-1" "S1_CTCATTAGTCAATACC-1" "S1_CTCGGGAGTTCACGGC-1"

In [ ]:
# We will edit the barcode names in the VDJ table to match the Seurat object
contig.list[[1]]$barcode = gsub("S-CoV1_M0_Sort1","S1",contig.list[[1]]$barcode)
contig.list[[1]]$barcode = gsub("S-CoV1_M6_Sort1","S2",contig.list[[1]]$barcode)
contig.list[[1]]$barcode = gsub("S-CoV13_M0_Sort1","S3",contig.list[[1]]$barcode)
contig.list[[1]]$barcode = gsub("S-CoV13_M6_Sort1","S4",contig.list[[1]]$barcode)

In [ ]:
# Check the barcode names in the contig list again (they should be changed to look like the barcode names in the Seurat object)
head(contig.list[[1]]$barcode,n=100)

[1] "S1_AGCTCTCGTTTGCATG-1" "S1_ATAGACCCATGTCGAT-1" "S1_ATCTGCCCAGCCAATT-1"
  [4] "S1_CAAGGCCGTGTGAAAT-1" "S1_CTCATTAGTCAATACC-1" "S1_CTCGGGAGTTCACGGC-1"
  [7] "S1_AACTCCCTCGTATCAG-1" "S2_ACATCAGTCGTGGACC-1" "S1_GACGCGTCATCGATGT-1"
 [10] "S1_CGAACATAGCAACGGT-1" "S3_CGGTTAATCGGCTACG-1" "S1_GTATTCTAGAGTCGGT-1"
 [13] "S1_AGGCCACCATCGGACC-1" "S1_GATCGATTCGAATGCT-1" "S1_GGGACCTAGTGCCAGA-1"
 [16] "S1_ACCCACTCATATGGTC-1" "S1_AGACGTTAGGGCTCTC-1" "S1_GCATGTAAGATGTAAC-1"
 [19] "S1_GCTTCCAGTGGCCCTA-1" "S4_TAGTTGGAGCGACGTA-1" "S1_CATATGGTCAGGCGAA-1"
 [22] "S1_CATCGAAAGATCTGAA-1" "S3_CCCAATCGTACCGCTG-1" "S4_GAAACTCCAGTGGAGT-1"
 [25] "S1_CATATTCTCGGAGGTA-1" "S1_GCAATCACAGGACGTA-1" "S1_GCTTGAACAAGCTGTT-1"
 [28] "S1_TAAACCGTCGCAGGCT-1" "S1_AGTGTCATCAGTGTTG-1" "S1_CCGGTAGAGAACTCGG-1"
 [31] "S1_GGGAGATAGAAACCAT-1" "S1_CGCCAAGGTTTAGCTG-1" "S1_AACCATGAGGTCATCT-1"
 [34] "S1_GCAGCCACATATACCG-1" "S1_GTCCTCAAGATGCGAC-1" "S1_ACGTCAAAGTACATGA-1"
 [37] "S1_AGCGGTCTCTTACCTA-1" "S1_ACGCCAGGTTACGGAG-1" "S1_TTGCGTCTCAACCATG-1"
 [40] "S1_GTGCGGTAGCTCTCGG-1" "S4_CTGTTTAAGCGCCTTG-1" "S1_GGAATAAGTGATAAGT-1"
 [43] "S2_GAATAAGTCTGTCAAG-1" "S4_CATCAAGCATTTGCTT-1" "S4_CGAGCACGTACAGTGG-1"
 [46] "S1_CAGAATCCACTACAGT-1" "S1_GACAGAGAGGTCATCT-1" "S1_TGACGGCCAGGGTATG-1"
 [49] "S1_CACAAACTCCTGTACC-1" "S3_GCTGCGATCGTTGCCT-1" "S1_AAAGTAGGTAGCAAAT-1"
 [52] "S1_TAGACCAGTTCACGGC-1" "S1_AGTTGGTAGGGTGTTG-1" "S1_TCTTTCCTCTTTACAC-1"
 [55] "S1_CTTCTCTAGTGTTGAA-1" "S1_CAGCTGGCAAACCTAC-1" "S1_CATTCGCCACACAGAG-1"
 [58] "S1_CGACTTCTCGCAAACT-1" "S1_CTAGTGACACGAGAGT-1" "S1_ACAGCTAAGATCTGCT-1"
 [61] "S1_CCACGGAGTTGAGGTG-1" "S1_GTCCTCACAACGATGG-1" "S1_AGCGGTCAGTGAACGC-1"
 [64] "S1_CCTATTAGTGCCTGCA-1" "S1_CGATCGGAGGCTCTTA-1" "S1_GACGCGTTCTGGTGTA-1"
 [67] "S3_CACCACTCAAGAAGAG-1" "S1_CAAGGCCGTCTAACGT-1" "S1_GCTCTGTTCTGCTTGC-1"
 [70] "S1_GCTGCTTAGTGAAGTT-1" "S1_TCTTTCCTCTTTACAC-1" "S1_GACACGCCATCATCCC-1"
 [73] "S1_TGAGAGGAGGAGTAGA-1" "S1_AATCGGTCAAGCTGAG-1" "S1_GAACCTAAGCGTCAAG-1"
 [76] "S1_GACGTGCTCGTACCGG-1" "S1_GGACAGATCGCAGGCT-1" "S1_TACTTGTAGGCGACAT-1"
 [79] "S1_CGATTGATCACAACGT-1" "S1_GTCCTCACATCCAACA-1" "S1_TGTGGTAGTCTGCGGT-1"
 [82] "S1_AAAGCAACATGTAGTC-1" "S1_TCCCGATAGACTCGGA-1" "S3_CAACTAGGTGTCGCTG-1"
 [85] "S1_CTCACACGTTTCCACC-1" "S1_GCAGCCACATATACCG-1" "S1_ATCTGCCCAGCCAATT-1"
 [88] "S1_ACGATGTAGTACGATA-1" "S1_CGACTTCAGTCCGGTC-1" "S1_TCAATCTTCGGCGGTT-1"
 [91] "S2_CTACGTCGTTATGCGT-1" "S1_CTACCCATCAACGAAA-1" "S1_GCAAACTCATTGGCGC-1"
 [94] "S1_GCGCAGTTCGGACAAG-1" "S1_GACGTGCTCGTACCGG-1" "S1_GTAGGCCTCCATTCTA-1"
 [97] "S1_GCGCAGTTCGGACAAG-1" "S1_CGTCACTAGATGTGTA-1" "S1_AAAGCAACATGTAGTC-1"
[100] "S1_TAAGCGTCAAAGAATC-1"

In [ ]:
# Now we have matching names between the Seurat object and the BCR VDJ contigs
# One last problem is that we have too many cells to analyze in Google colab. 25 thousand is a lot, so we will sample down to 5 thousand to speed up our analyses.
# We have to be careful to sample the same cells from the Seurat object and the contigs file.
bcell_sample_ids = sample(colnames(bcells),5000,replace=F)
bcells@meta.data$CellName = colnames(bcells)
bcells = subset(bcells, subset = CellName %in% bcell_sample_ids)

In [ ]:
# Sample the contig list to a the cells we sampled for the Seurat object
contig.list[[1]] = contig.list[[1]][contig.list[[1]]$barcode %in% bcell_sample_ids,]
dim(contig.list[[1]])

[1] 6901    9

In [ ]:
# Finally we can run combineBCR to make cell annotations and call clones
combined.BCR <- combineBCR(contig.list, samples = "P1", threshold = 0.85)

In [ ]:
# We have to clean up these barcode names after the combineBCR funtion added "P1_" to each name
combined.BCR$P1$barcode = sub("P1_","",combined.BCR$P1$barcode)

In [ ]:
# Now we can integrate the gene expression in the Seurat object with the BCR calls
bcells = combineExpression(combined.BCR,bcells,cloneCall="gene")

In [ ]:
# Check the object identities
Idents(object = bcells) <- "patient"
table(Idents(bcells))


 S-CoV1 S-CoV13 
   2700    2300 

From this point please continue to process the data if necessary and perform analysis to answer the questions described at the top of the notebook. Good luck!